## 1. Import Libraries and Set Up

In [ ]:
!pip install opencv-python

In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights -O yolov3.weights
!wget https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg -O yolov3.cfg
!wget https://github.com/pjreddie/darknet/blob/master/data/coco.names -O coco.names
!ls -lh yolov3.weights yolov3.cfg coco.names

In [ ]:
!wget https://github.com/pjreddie/darknet/raw/master/cfg/yolov3.cfg -O yolov3.cfg

In [ ]:
# Import necessary libraries
import cv2
import numpy as np
import time
from google.colab.patches import cv2_imshow

## 2. Vehicle Detection Script

In [ ]:
# Cell 1: Vehicle Detection

# Load YOLO pre-trained model for vehicle detection
net = cv2.dnn.readNet("/content/yolov3.weights", "/content/yolov3.cfg")

# Load the COCO names file (contains class names)
classes = []
with open("coco.names", "r") as f:
    classes = f.read().strip().split("\n")

# Load the image
image_path = "/content/car1.jpeg"  # Ensure this image is uploaded
image = cv2.imread(image_path)

# Get image dimensions
height, width, _ = image.shape

# Preprocess the image for YOLO
blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

# Set input blob for the network
net.setInput(blob)

# Get output layer names
output_layers = net.getUnconnectedOutLayersNames()

# Perform forward pass and get detections
detections = net.forward(output_layers)

# Loop through detections
for detection in detections:
    for obj in detection:
        scores = obj[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        if confidence > 0.5 and class_id == 2:  # Class ID for cars is 2
            center_x = int(obj[0] * width)
            center_y = int(obj[1] * height)
            w = int(obj[2] * width)
            h = int(obj[3] * height)

            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(image, "Vehicle", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display the image with detections
cv2_imshow(image)

In [ ]:
# Cell 1: Vehicle Detection

import cv2
import numpy as np

# Load YOLO pre-trained model for vehicle detection
net = cv2.dnn.readNet("/content/yolov3.weights", "/content/yolov3.cfg")

# Load the COCO names file (contains class names)
classes = []
with open("coco.names", "r") as f:
    classes = f.read().strip().split("\n")

# Load the image
image_path = "/content/car1.jpeg"  # Ensure this image is uploaded
image = cv2.imread(image_path)

# Get image dimensions
height, width, _ = image.shape

# Preprocess the image for YOLO
blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

# Set input blob for the network
net.setInput(blob)

# Get output layer names
output_layers = net.getUnconnectedOutLayersNames()

# Perform forward pass and get detections
detections = net.forward(output_layers)

# Initialize lists for bounding boxes, confidences, and class IDs
boxes = []
confidences = []
class_ids = []

# Loop through detections
for detection in detections:
    for obj in detection:
        scores = obj[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        if confidence > 0.5 and class_id == 2:  # Class ID for cars is 2
            center_x = int(obj[0] * width)
            center_y = int(obj[1] * height)
            w = int(obj[2] * width)
            h = int(obj[3] * height)

            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            # Add to lists
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Apply Non-Maximum Suppression to remove overlapping bounding boxes
indices = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.4)

# Draw bounding boxes for detections that are kept after NMS
for i in indices.flatten():
    x, y, w, h = boxes[i]
    label = str(classes[class_ids[i]])
    confidence = confidences[i]

    # Draw rectangle around detected car
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(image, f"{label} {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display the image with detections
cv2_imshow(image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 3. Capture Image Script

In [ ]:
# Cell 2: Capture Image

def capture_image(camera_index, output_path):
    # Open the camera
    cap = cv2.VideoCapture(camera_index)

    # Check if the camera opened successfully
    if not cap.isOpened():
        print("Error: Could not open camera.")
        return

    # Read a frame from the camera
    ret, frame = cap.read()

    # Check if the frame was read successfully
    if not ret:
        print("Error: Could not read frame from camera.")
        return

    # Save the captured frame as an image
    cv2.imwrite(output_path, frame)

    # Release the camera
    cap.release()
    print("Image captured and saved as", output_path)

def main_capture():
    camera_index = 0  # Change this to the appropriate camera index
    interval_seconds = 5  # Time interval between captures

    for _ in range(3):  # Capture 3 images for demonstration
        timestamp = time.strftime("%Y%m%d%H%M%S")
        image_filename = f"captured_image_{timestamp}.jpg"

        capture_image(camera_index, image_filename)
        time.sleep(interval_seconds)

main_capture()


## 4. Adjust Green Signal Time Script

In [ ]:
# Cell 3: Adjust Green Signal Time

def adjust_green_signal_time(vehicle_count):
    base_green_time = 30  # Base green time in seconds
    vehicle_multiplier = 2  # Green time increases by 2 seconds per vehicle

    green_time = base_green_time + (vehicle_count * vehicle_multiplier)
    return green_time

def main_adjust():
    try:
        with open("vehicle_count.txt", "r") as file:
            vehicle_count = int(file.read())

            if vehicle_count < 0:
                print("Invalid vehicle count in the file. Please ensure the count is a non-negative integer.")
                return

        green_time = adjust_green_signal_time(vehicle_count)
        print("Adjusted Green Signal Time:", green_time, "seconds")
    except (ValueError, FileNotFoundError):
        print("Error reading vehicle count from file.")

main_adjust()
